# Error Correction

220604

- [blog](https://towardsdatascience.com/nlp-building-a-grammatical-error-correction-model-deep-learning-analytics-c914c3a8331b)
- [code](https://github.com/priya-dwivedi/Deep-Learning/tree/master/GrammarCorrector)
- [data](https://github.com/google-research-datasets/C4_200M-synthetic-dataset-for-grammatical-error-correction)
- [c4 data paper](https://aclanthology.org/2021.bea-1.4.pdf)
- [정제된 데이터](https://drive.google.com/drive/folders/1kKlGcinD_FhGXC0LztN4Ts605YXzMEVA)

# Prepare

In [4]:
## install
!pip install datasets tqdm pandas
!pip install sentencepiece==0.1.90
!pip install transformers==4.16.0
# !pip install torch==1.10
!pip install wandb
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 29.7 MB/s 
     |████████████████████████████████| 880 kB 56.9 MB/s 
     |████████████████████████████████| 6.6 MB 60.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=88434f00df88cbb8382a5aaa3052379d8e465feec32bab4e65cdb12c90690976
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
## load library
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, load_metric
rouge_metric = load_metric("rouge")

from transformers import (T5ForConditionalGeneration, T5Tokenizer,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          DataCollatorForSeq2Seq, AdamW,
                          get_linear_schedule_with_warmup)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


set_seed(42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
## load model 
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [8]:
## load dataset
pd.set_option('display.max_colwidth', None)

# DATA_ROOT = 'data'
# data_name = 'c4_200m_550k.csv'  # 55만개
# data_name = 'c4_200m_1M.csv'  # 100만개

DATA_ROOT = './'
data_name = '2_grammer_error_dataset.csv'  # 100만개
data_path = os.path.join(DATA_ROOT, data_name)
df = pd.read_csv(data_path).dropna(axis=0)
print(df.shape)
df.head()

(99998, 2)


,correct_sentence,error_sentence
0,Answers Regions Is Nagorno Karabakh region part of Armenia,Answers Regions Is Nagorno Karabakh region part at Armenia
1,Flaneuring Fun in Maple Creek SK,Flaneurg Fun Maple Creek SK
2,About Private Investigators Ellesmere Port In Ellesmere Port Cheshire,About PEivate InvestigatoEs EllesmeEe PoEt In EllesmeEe PoEt PoEt CheshiEe
3,Bake in the oven for 35 mins scattering the flaked almonds after the first 20,Bake in the oven for 35 mins scattering the the flaked almonds after the 20
4,informing you of changes in our web site,formg you by changes our web site


In [9]:
## make data
# 0.5%만(5,000개) test, 99.5%는 학습
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.005, shuffle=True)
print(train_df.shape, test_df.shape)

test_df['input_token_len'] = test_df['error_sentence'].apply(calc_token_len)
test_df.head()

(99498, 2) (500, 2)


,correct_sentence,error_sentence,input_token_len
33969,Out of interest is that classed as restriction of trade,Out interest is that classed as restriction trade,11
24612,We must move forward grow evolve adapt if we are to survive,We must move forward grow evolve adapt if we are survive,13
42678,New Lush bubble bar dropping soon,NNw Nush bubblN bar dropping soon,11
9732,Free Online Bible Study Course Easy and basic Bible study course,Free Online Bible Study Course Easy basic Bible study course,11
7129,Re Need help with Chinese,Re Need help Chinese,5


In [10]:
test_df['input_token_len'].describe()

count    500.000000
mean      13.770000
std        5.428447
min        5.000000
25%       10.000000
50%       13.000000
75%       16.000000
max       39.000000
Name: input_token_len, dtype: float64

In [11]:
# df to dataset
# from datasets import Dataset
from datasets import Dataset as datasets_Dataset
train_dataset = datasets_Dataset.from_pandas(train_df)
test_dataset = datasets_Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['correct_sentence', 'error_sentence', 'input_token_len', '__index_level_0__'],
    num_rows: 500
})

In [12]:
# custom dataset
from torch.utils.data import Dataset
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer, print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)

    def tokenize_data(self, example):
        input_, target_ = example['error_sentence'], example['correct_sentence']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_,
                                     pad_to_max_length=self.pad_to_max_length,
                                     max_length=self.max_len,
                                     return_attention_mask=True)

        tokenized_targets = tokenizer(target_,
                                      pad_to_max_length=self.pad_to_max_length,
                                      max_length=self.max_len,
                                      return_attention_mask=True)

        inputs = {
            "input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs

    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [13]:
# check dataset
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 7
attention_mask 7
labels 6
{'input_ids': [11419, 11419, 19, 82, 793, 11341, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1], 'labels': [11419, 19, 82, 793, 11341, 1]}


# Train Model

In [14]:
## config
# defining training related arguments
batch_size = 10
args = Seq2SeqTrainingArguments(
    output_dir='weights',
    evaluation_strategy='steps',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=True,
    gradient_accumulation_steps=6,
    eval_steps=50,
    save_steps=100,
    load_best_model_at_end=True,
    logging_dir='logs',
    report_to='wandb')

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [15]:
# metric
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions,
                                           skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip()))
        for label in decoded_labels
    ]

    result = rouge_metric.compute(predictions=decoded_preds,
                                  references=decoded_labels,
                                  use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id)
        for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [16]:
# trainer
trainer = Seq2SeqTrainer(model=model,
                         args=args,
                         train_dataset=GrammarDataset(train_dataset, tokenizer),
                         eval_dataset=GrammarDataset(test_dataset, tokenizer),
                         tokenizer=tokenizer,
                         data_collator=data_collator,
                         compute_metrics=compute_metrics)

Using amp half precision backend


# Train!!

In [17]:
## train!!
trainer.train()
trainer.save_model('t5_gec_model')

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 99498
  Num Epochs = 1
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 60
  Gradient Accumulation steps = 6
  Total optimization steps = 1658
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
20,No log,1.607820,62.845800,50.047700,61.178800,61.877600,17.672000
40,No log,1.244085,74.456100,60.547300,73.737900,73.764900,14.252000
60,No log,1.104428,78.905400,64.770200,78.256600,78.209900,13.060000
80,No log,1.004348,80.436300,66.691400,80.048900,80.034300,12.962000
100,No log,0.942424,81.935400,68.414200,81.545200,81.586400,13.028000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 10
Saving model checkpoint to weights/checkpoint-20
Configuration saved in weights/checkpoint-20/config.json
Model weights saved in weights/checkpoint-20/pytorch_model.bin
tokenizer config file saved in weights/checkpoint-20/tokenizer_config.json
Special tokens file saved in weights/checkpoint-20/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 10
Saving model checkpoint to weights/checkpoint-40
Configuration saved in weights/checkpoint-40/config.json
Model weights saved in weights/checkpoint-40/pytorch_model.bin
tokenizer config file saved in weights/checkpoint-40/tokenizer_config.json
Special tokens file saved in weights/checkpoint-40/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 10
Saving model checkpoint to weights/checkpoint-60
Configuration saved in weights/checkpoint-60/config.json
Model weights saved in weights/checkpoint-60/p

KeyboardInterrupt: ignored

# Inference

In [18]:
# prepare inference
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
# model_name = 'deep-learning-analytics/GrammarCorrector'
# model_name = 't5_gec_model'
model_name = 'weights/checkpoint-100'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
print('load model done')

def correct_grammar(input_text, num_return_sequences):
    batch = tokenizer([input_text],
                      truncation=True,
                      padding='max_length',
                      max_length=64,
                      return_tensors="pt").to(device)
    translated = model.generate(**batch,
                                max_length=64,
                                num_beams=4,
                                num_return_sequences=num_return_sequences,
                                temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

Didn't find file weights/checkpoint-100/added_tokens.json. We won't load it.
Didn't find file weights/checkpoint-100/tokenizer.json. We won't load it.
loading file weights/checkpoint-100/spiece.model
loading file None
loading file weights/checkpoint-100/special_tokens_map.json
loading file weights/checkpoint-100/tokenizer_config.json
loading file None
loading configuration file weights/checkpoint-100/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    

load model done


### Do inference!!

In [19]:
text = 'He are moving here.'
print(correct_grammar(text, num_return_sequences=2))

['He are moving here', 'He are moving here.']


In [20]:
text = 'Cat drinked milk'
print(correct_grammar(text, num_return_sequences=2))

['Cat drinked milk', 'Cat drinked milk. Cat drinked milk.']


# Done!